# ベンチマークデータセットのGCN

In [1]:
pip install seaborn

     |████████████████████████████████| 285 kB 1.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import networkx as nx
import torch
import numpy as np
import pandas as pd
import sys
sys.path.append("../codes")

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import TUDataset

from model import GCN
from train_test import train, test
from my_data_train_utils import conf_matrix

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
# data load and split

names = ["MUTAG", "DD", "REDDIT-BINARY", "COLLAB"]
names = ["COLLAB"]
for data_name in names:
    dataset = TUDataset(root='data/TUDataset', name=data_name)

    torch.manual_seed(12345)
    dataset = dataset.shuffle()

    train_index = int(0.7 * len(dataset))

    train_dataset = dataset[:train_index]
    test_dataset = dataset[train_index:]

    train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

    # set model and train
    model = GCN(hidden_channels=64, dataset=dataset)
    model.to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#     criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adagrad(model.parameters(), lr=0.05, eps=1e-5)
    criterion = torch.nn.CrossEntropyLoss()
    train_accs = []
    test_accs = []


    for epoch in range(1, 20):
        train(model, train_loader, criterion, optimizer, device, x="degree")
        train_acc = test(model, train_loader, device, x="degree")
        test_acc = test(model, test_loader, device, x="degree")

        train_accs.append(train_acc)
        test_accs.append(test_acc)

        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

    # log を保存する
    save_dir = "data/TUDataset/{}/result".format(data_name)
    """log の保存"""
    # 保存先ディレクトリの作成
    print(save_dir)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # model(重み) の保存
    torch.save(model.to("cpu").state_dict(), save_dir + "/model.pth")
    # 学習結果の保存

    df = pd.DataFrame()
    df["train_acc"] = train_accs
    df["test_acc"] = test_accs
    df.to_csv(save_dir + "/log.csv")
    # 学習曲線の保存
    plt.style.use('ggplot')
    plt.plot(train_accs, label="train")
    plt.plot(test_accs, label="test")
    plt.ylim(0,1)
    plt.legend()
    plt.savefig(save_dir + "/learning.png")
    plt.clf()
    plt.close()
    
    # make conf matrix
    conf_matrix(model, test_loader, device, benchmark=True, benchmark_class=dataset.num_classes, x="degree")
    plt.savefig(save_dir + "/conf_matrix.png")
    plt.clf()
    plt.close()

Epoch: 001, Train Acc: 0.6471, Test Acc: 0.6420
Epoch: 002, Train Acc: 0.6423, Test Acc: 0.6333
Epoch: 003, Train Acc: 0.6494, Test Acc: 0.6440
Epoch: 004, Train Acc: 0.6480, Test Acc: 0.6387
Epoch: 005, Train Acc: 0.6780, Test Acc: 0.6693
Epoch: 006, Train Acc: 0.6694, Test Acc: 0.6580
Epoch: 007, Train Acc: 0.6609, Test Acc: 0.6487
Epoch: 008, Train Acc: 0.6803, Test Acc: 0.6707
Epoch: 009, Train Acc: 0.6766, Test Acc: 0.6640
Epoch: 010, Train Acc: 0.6763, Test Acc: 0.6620
Epoch: 011, Train Acc: 0.6740, Test Acc: 0.6620
Epoch: 012, Train Acc: 0.6769, Test Acc: 0.6700
Epoch: 013, Train Acc: 0.6866, Test Acc: 0.6767
Epoch: 014, Train Acc: 0.6711, Test Acc: 0.6627
Epoch: 015, Train Acc: 0.6660, Test Acc: 0.6567
Epoch: 016, Train Acc: 0.6806, Test Acc: 0.6700
Epoch: 017, Train Acc: 0.6611, Test Acc: 0.6507
Epoch: 018, Train Acc: 0.6743, Test Acc: 0.6667
Epoch: 019, Train Acc: 0.6811, Test Acc: 0.6747
data/TUDataset/COLLAB/result


In [7]:
data_iter = iter(train_loader)

data = data_iter.next()

In [9]:
data.x

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])